# CS 5665 Final Project

Data files loaded previously into local repository.

# Baseline NN Model

Load Libraries/Data

In [3]:
import numpy as np
import pandas as pd
import json
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from google.colab import files
# device = 'cuda'
# uploaded = files.upload()

Hyperparameters

In [4]:
N_EPOCH = 10
BATCH_SIZE = 128
LEARN_RATE = 0.01
FEATURE_SIZE = 21

Encoding Functions

In [5]:
def one_hot(categories, string):
    encoding = np.zeros((len(string), len(categories)))
    for idx, char in enumerate(string):
        encoding[idx, categories.index(char)] = 1
    return encoding

def featurize(entity):
    sequence = one_hot(list('ACGU'), entity['sequence'])
    structure = one_hot(list('.()'), entity['structure'])
    loop_type = one_hot(list('BEHIMSX'), entity['predicted_loop_type'])
    features = np.hstack([sequence, structure, loop_type])
    return features 

def char_encode(index, features, feature_size):
    half_size = (feature_size - 1) // 2
    
    if index - half_size < 0:
        char_features = features[:index+half_size+1]
        padding = np.zeros((int(half_size - index), char_features.shape[1]))
        char_features = np.vstack([padding, char_features])
    elif index + half_size + 1 > len(features):
        char_features = features[index-half_size:]
        padding = np.zeros((int(half_size - (len(features) - index))+1, char_features.shape[1]))
        char_features = np.vstack([char_features, padding])
    else:
        char_features = features[index-half_size:index+half_size+1]
    
    return char_features

Datasets

In [6]:
class VaxDataset(Dataset):
    def __init__(self, path, test=False):
        self.path = path
        self.test = test
        self.features = []
        self.targets = []
        self.ids = []
        self.load_data()
    
    def load_data(self):
        with open(self.path, 'r') as text:
            for line in text:
                records = json.loads(line)
                features = featurize(records)
                
                for char_i in range(records['seq_scored']):
                    char_features = char_encode(char_i, features, FEATURE_SIZE)
                    self.features.append(char_features)
                    self.ids.append('%s_%d' % (records['id'], char_i))
                        
                if not self.test:
                    targets = np.stack([records['reactivity'], records['deg_Mg_pH10'], records['deg_Mg_50C']], axis=1)
                    self.targets.extend([targets[char_i] for char_i in range(records['seq_scored'])])
                    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, index):
        if self.test:
            return self.features[index], self.ids[index]
        else:
            return self.features[index], self.targets[index], self.ids[index]

## Training Data

In [7]:
train_dataset = VaxDataset('train.json')
train_dataloader = DataLoader(train_dataset, BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)

Model

In [8]:
class Flatten(nn.Module):
    def forward(self, x):
        batch_size = x.shape[0]
        return x.view(batch_size, -1)

class VaxModel(nn.Module):
    def __init__(self):
        super(VaxModel, self).__init__()
        self.layers = nn.Sequential(
            nn.Dropout(0.3),
            nn.Conv1d(14, 32, 1, 1),
            nn.ReLU(),
            nn.BatchNorm1d(32),
            nn.Dropout(0.3),
            nn.Conv1d(32, 1, 1, 1),
            nn.ReLU(),
            Flatten(),
            nn.Dropout(0.2),
            nn.Linear(FEATURE_SIZE, 32),
            nn.ReLU(),
            nn.BatchNorm1d(32),
            nn.Dropout(0.3),
            nn.Linear(32, 3),
        )
    
    def forward(self, features):
        return self.layers(features)

Train the model

In [9]:
model = VaxModel().cuda()
optimizer = torch.optim.Adam(model.parameters(), LEARN_RATE)
criterion = nn.MSELoss()

In [10]:
for epoch in range(N_EPOCH):
    losses = []
    model.train()
    for features, targets, ids in train_dataloader:
        features = features.cuda().permute(0,2,1).float()
        targets = targets.cuda().float()
        predictions = model(features)
        loss = criterion(predictions, targets)
        for p in model.parameters():
            p.grad = None
        loss.backward()
        optimizer.step()
        losses.append(loss.detach().cpu().numpy())
    avg_loss = float(np.mean(losses))
    print(epoch, avg_loss)

torch.save(model.state_dict(), 'weights.pth')

0 0.5535919070243835
1 0.5437663793563843
2 0.5426644682884216
3 0.5410770177841187
4 0.5407720804214478
5 0.5422633290290833
6 0.539517343044281
7 0.5399692058563232
8 0.5412024259567261
9 0.5402473211288452


## Testing Data

Configuration and Data Loading

In [11]:
WEIGHTS_PATH = 'weights.pth'
FEATURE_SIZE = 21
BATCH_SIZE = 128

In [12]:
test_dataset = VaxDataset('test.json', test=True)
test_dataloader = DataLoader(test_dataset, BATCH_SIZE, num_workers=4, drop_last=False, pin_memory=True)

Inference

In [13]:
model = VaxModel().cuda()

In [14]:
sub = pd.read_csv('sample_submission.csv', index_col='id_seqpos')

In [15]:
from tqdm import tqdm
model.load_state_dict(torch.load(WEIGHTS_PATH))
model.eval()
for features, ids in tqdm(test_dataloader):
    features = features.cuda().permute(0,2,1).float()
    predictions = model(features)
    sub.loc[ids, ['reactivity', 'deg_Mg_pH10', 'deg_Mg_50C']] = predictions.detach().cpu().numpy()

100%|██████████| 2471/2471 [00:09<00:00, 253.68it/s]


Submission CSV

In [16]:
sub.to_csv('submission3.csv')

# Resnet18 Model

In [17]:
import torch.nn as nn
def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.conv1)
    # according to "Deep residual learning for image recognition"https://arxiv.org/abs/1512.03385.
    # This variant is also known as ResNet V1.5 and improves accuracy according to
    # https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch.

    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(1, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _forward_impl(self, x):
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x):
        return self._forward_impl(x)


def _resnet(arch, block, layers, pretrained, progress, **kwargs):
    model = ResNet(block, layers, **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model


def resnet18(pretrained=False, progress=True, **kwargs):
    r"""ResNet-18 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet18', BasicBlock, [1, 1, 1, 1], pretrained, progress,
                   **kwargs)


class VaxModel(nn.Module):

    def __init__(self):
        super(VaxModel, self).__init__()
        self.resnet = resnet18()
        self.drop = nn.Dropout(0.3)
        self.linear1 =   nn.Linear(1000, 32)
        self.drop1 = nn.Dropout(0.3)
        self.rl1 =    nn.PReLU()
        self.btn1=   nn.BatchNorm1d(32)
        self.drop2=  nn.Dropout(0.3)
        self.ln =     nn.Linear(32, 3)
        

    def forward(self, x):
        x = self.drop(x)
        x = x.reshape(x.shape[0],1, x.shape[1], x.shape[2])
        x = self.resnet(x)
        x = self.drop1(x)
        x = self.linear1(x)
        x = self.rl1(x)
        x = self.btn1(x)
        x = self.drop2(x)
        x = self.ln(x)
        return x

Training Data

In [18]:
model = VaxModel().cuda()
optimizer = torch.optim.Adam(model.parameters(), LEARN_RATE)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',patience=5,factor=0.6)
criterion = nn.MSELoss()

In [19]:
best_avg = 1
stop_ep = 0
for epoch in range(N_EPOCH):
    losses = []
    model.train()
    for features, targets, ids in train_dataloader:
        features = features.cuda().permute(0,2,1).float()
        targets = targets.cuda().float()
        predictions = model(features)
        loss = criterion(predictions, targets)
        for p in model.parameters():
            p.grad = None
        loss.backward()
        optimizer.step()
        losses.append(loss.detach().cpu().numpy())
    avg_loss = float(np.mean(losses))
    scheduler.step(metrics= avg_loss)
    if avg_loss < best_avg:
        stop_ep = 0
        best_avg = avg_loss
        print(f"{epoch}: {avg_loss} saved model!!!!")
        torch.save(model.state_dict(), 'weights.pth')
    else:
        stop_ep += 1
        print(f"{epoch}: {avg_loss}")
    if stop_ep >= 10:
        break

0: 0.5291779637336731 saved model!!!!
1: 0.5043544173240662 saved model!!!!
2: 0.5013427734375 saved model!!!!
3: 0.49684062600135803 saved model!!!!
4: 0.496849924325943
5: 0.49297717213630676 saved model!!!!
6: 0.49234914779663086 saved model!!!!
7: 0.48822274804115295 saved model!!!!
8: 0.48568522930145264 saved model!!!!
9: 0.48500001430511475 saved model!!!!


Testing Data

In [20]:
model = VaxModel().cuda()

In [21]:
sub = pd.read_csv('sample_submission.csv', index_col='id_seqpos')

In [22]:
model.load_state_dict(torch.load(WEIGHTS_PATH))
model.eval()
for features, ids in tqdm(test_dataloader):
    features = features.cuda().permute(0,2,1).float()
    predictions = model(features)
    sub.loc[ids, ['reactivity', 'deg_Mg_pH10', 'deg_Mg_50C']] = predictions.detach().cpu().numpy()

100%|██████████| 2471/2471 [00:15<00:00, 159.18it/s]


In [23]:
sub.to_csv('submission4.csv')